# API FOOTBALL

## Estrazione Squadre e Giocatori da una Competizione

In questo notebook, useremo l'API di API-SPORTS per ottenere tutte le squadre e i giocatori di una competizione. In particolare, ci focalizzeremo sulla Premier League inglese per la stagione 2021-2022.

Dall'esempio [how-to-get-all-teams-and-players-from-a-league-id](https://www.api-football.com/news/post/how-to-get-all-teams-and-players-from-a-league-id)

In [46]:
# Importare le Librerie
import requests
import time
import json
import pandas as pd

In [50]:
from dotenv import load_dotenv
import os

# Carica le variabili d'ambiente dal file .env
load_dotenv()
RAPIDAPI_KEY = os.environ['API_KEY']


In [51]:
# Funzioni per chiamare l'API
def call_api(endpoint, params=None):
    url = f'https://v3.football.api-sports.io/{endpoint}'
    headers = {
        'x-rapidapi-key': RAPIDAPI_KEY 
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()


In [36]:
# funzione ricorsiva per ottenere i dati dei giocatori
def fetch_players_data(league, season, page=1, players_data=[]):
    params = {
        'league': league,
        'season': season,
        'page': page
    }
    players = call_api('players', params)
    players_data.extend(players['response'])

    if players['paging']['current'] < players['paging']['total']:
        page += 1
        if page % 2 == 1:
            time.sleep(1)  # Pausa di 1 secondo per evitare di superare il rate limit
        return fetch_players_data(league, season, page, players_data)
    
    return players_data

In [33]:
# Funzione per ottenere i dati dei giocatori da una specifica lega e stagione
# def fetch_players_data(league, season, page=1, players_list=[]):
#     url = f"https://api-football-v1.p.rapidapi.com/v3/players"
#     querystring = {"league": league, "season": season, "page": page}
#     headers = {
#         'x-rapidapi-key': RAPIDAPI_KEY
#     }

#     response = requests.request("GET", url, headers=headers, params=querystring)
#     data = response.json()

#     if data['results'] == 0:
#         return players_list

#     players_list.extend(data['response'])

#     if page % 2 == 1:
#         time.sleep(1)  # Pausa di 1 secondo per evitare di superare il rate limit

#     return fetch_players_data(league, season, page + 1, players_list)

### Estrazione delle squadre

In [11]:
# Ottenere Tutte le Squadre della Competizione
# Utilizziamo la funzione `call_api` per ottenere tutte le squadre della Serie A per la stagione 2023-2024
# parametri: league_id = 135, season = 2023

teams = call_api('teams', {'league': 135, 'season': 2023})
print(json.dumps(teams, indent=4))

{
    "get": "teams",
    "parameters": {
        "league": "135",
        "season": "2023"
    },
    "errors": [],
    "results": 20,
    "paging": {
        "current": 1,
        "total": 1
    },
    "response": [
        {
            "team": {
                "id": 487,
                "name": "Lazio",
                "code": "LAZ",
                "country": "Italy",
                "founded": 1900,
                "national": false,
                "logo": "https://media.api-sports.io/football/teams/487.png"
            },
            "venue": {
                "id": 910,
                "name": "Stadio Olimpico",
                "address": "Viale dei Gladiatori, 2 / Via del Foro Italico",
                "city": "Roma",
                "capacity": 68530,
                "surface": "grass",
                "image": "https://media.api-sports.io/football/venues/910.png"
            }
        },
        {
            "team": {
                "id": 488,
                "name": "Sa

In [27]:
# Converto il json in un DF per esplorazione
# Estrarre la parte di response
teams_response = teams['response']

# Creare una lista di dizionari con le informazioni della squadra
teams_list = []
for team in teams_response:
    team_info = {
        'team_id': team['team']['id'],
        'team_name': team['team']['name'],
        'team_code': team['team']['code'],
        'team_country': team['team']['country'],
        'team_founded': team['team']['founded'],
        'team_national': team['team']['national'],
        'team_logo': team['team']['logo'],
        'venue_id': team['venue']['id'],
        'venue_name': team['venue']['name'],
        'venue_address': team['venue']['address'],
        'venue_city': team['venue']['city'],
        'venue_capacity': team['venue']['capacity'],
        'venue_surface': team['venue']['surface'],
        'venue_image': team['venue']['image']
    }
    teams_list.append(team_info)

# Convertire la lista di dizionari in un DataFrame
teams_df = pd.DataFrame(teams_list)

# Visualizzare il DataFrame
teams_df

,team_id,team_name,team_code,team_country,team_founded,team_national,team_logo,venue_id,venue_name,venue_address,venue_city,venue_capacity,venue_surface,venue_image
0,487,Lazio,LAZ,Italy,1900,False,https://media.api-sports.io/football/teams/487...,910,Stadio Olimpico,"Viale dei Gladiatori, 2 / Via del Foro Italico",Roma,68530,grass,https://media.api-sports.io/football/venues/91...
1,488,Sassuolo,SAS,Italy,1922,False,https://media.api-sports.io/football/teams/488...,935,MAPEI Stadium - Città del Tricolore,"Piazza Azzuri d&apos;Italia, 1",Reggio Emilia,23717,grass,https://media.api-sports.io/football/venues/93...
2,489,AC Milan,MIL,Italy,1899,False,https://media.api-sports.io/football/teams/489...,907,Stadio Giuseppe Meazza,Via Piccolomini 5,Milano,80018,grass,https://media.api-sports.io/football/venues/90...
3,490,Cagliari,CAG,Italy,1920,False,https://media.api-sports.io/football/teams/490...,12275,Unipol Domus,Via Raimondo Carta Raspi,Cagliari,16416,grass,https://media.api-sports.io/football/venues/12...
4,492,Napoli,NAP,Italy,1904,False,https://media.api-sports.io/football/teams/492...,11904,Stadio Diego Armando Maradona,Pizzale Vincenzo Tecchio,Napoli,60240,grass,https://media.api-sports.io/football/venues/11...
5,494,Udinese,UDI,Italy,1896,False,https://media.api-sports.io/football/teams/494...,20416,Bluenergy Stadium,"Piazza le Repubblica Argentina, 3",Udine,25952,grass,https://media.api-sports.io/football/venues/20...
6,495,Genoa,GEN,Italy,1893,False,https://media.api-sports.io/football/teams/495...,905,Stadio Comunale Luigi Ferraris,"Via Giovanni De Prà, 1",Genova,36703,grass,https://media.api-sports.io/football/venues/90...
7,496,Juventus,JUV,Italy,1897,False,https://media.api-sports.io/football/teams/496...,909,Allianz Stadium,Strada Comunale di Altessano 131,Torino,45666,grass,https://media.api-sports.io/football/venues/90...
8,497,AS Roma,ROM,Italy,1927,False,https://media.api-sports.io/football/teams/497...,910,Stadio Olimpico,"Viale dei Gladiatori, 2 / Via del Foro Italico",Roma,68530,grass,https://media.api-sports.io/football/venues/91...
9,499,Atalanta,ATA,Italy,1907,False,https://media.api-sports.io/football/teams/499...,879,Gewiss Stadium,Viale Giulio Cesare 18,Bergamo,21300,grass,https://media.api-sports.io/football/venues/87...


### Estrazione dei Giocatori

In [37]:
players_list = fetch_players_data(135, 2023)

In [38]:
players_list_json = print(json.dumps(players_list, indent=4))
players_list_json

[
    {
        "player": {
            "id": 188,
            "name": "S. Handanovi\u010d",
            "firstname": "Samir",
            "lastname": "Handanovi\u010d",
            "age": 39,
            "birth": {
                "date": "1984-07-14",
                "place": "Ljubljana",
                "country": "Slovenia"
            },
            "nationality": "Slovenia",
            "height": "193 cm",
            "weight": "92 kg",
            "injured": false,
            "photo": "https://media.api-sports.io/football/players/188.png"
        },
        "statistics": [
            {
                "team": {
                    "id": 505,
                    "name": "Inter",
                    "logo": "https://media.api-sports.io/football/teams/505.png"
                },
                "league": {
                    "id": 135,
                    "name": "Serie A",
                    "country": "Italy",
                    "logo": "https://media.api-sports.io/football/

In [40]:
# Creare una lista di dizionari con tutte le informazioni dei giocatori
players_info_list = []
for player_entry in players_list:
    player = player_entry['player']
    for stat in player_entry['statistics']:
        player_info = {
            'player_id': player['id'],
            'player_name': player['name'],
            'player_firstname': player['firstname'],
            'player_lastname': player['lastname'],
            'player_age': player['age'],
            'player_birth_date': player['birth']['date'],
            'player_birth_place': player['birth']['place'],
            'player_birth_country': player['birth']['country'],
            'player_nationality': player['nationality'],
            'player_height': player['height'],
            'player_weight': player['weight'],
            'player_injured': player['injured'],
            'player_photo': player['photo'],
            'team_id': stat['team']['id'],
            'team_name': stat['team']['name'],
            'team_logo': stat['team']['logo'],
            'league_id': stat['league']['id'],
            'league_name': stat['league']['name'],
            'league_country': stat['league']['country'],
            'league_logo': stat['league']['logo'],
            'league_flag': stat['league']['flag'],
            'season': stat['league']['season'],
            'games_appearences': stat['games']['appearences'],
            'games_lineups': stat['games']['lineups'],
            'games_minutes': stat['games']['minutes'],
            'games_number': stat['games']['number'],
            'games_position': stat['games']['position'],
            'games_rating': stat['games']['rating'],
            'games_captain': stat['games']['captain'],
            'substitutes_in': stat['substitutes']['in'],
            'substitutes_out': stat['substitutes']['out'],
            'substitutes_bench': stat['substitutes']['bench'],
            'shots_total': stat['shots']['total'],
            'shots_on': stat['shots']['on'],
            'goals_total': stat['goals']['total'],
            'goals_conceded': stat['goals']['conceded'],
            'goals_assists': stat['goals']['assists'],
            'goals_saves': stat['goals']['saves'],
            'passes_total': stat['passes']['total'],
            'passes_key': stat['passes']['key'],
            'passes_accuracy': stat['passes']['accuracy'],
            'tackles_total': stat['tackles']['total'],
            'tackles_blocks': stat['tackles']['blocks'],
            'tackles_interceptions': stat['tackles']['interceptions'],
            'duels_total': stat['duels']['total'],
            'duels_won': stat['duels']['won'],
            'dribbles_attempts': stat['dribbles']['attempts'],
            'dribbles_success': stat['dribbles']['success'],
            'dribbles_past': stat['dribbles']['past'],
            'fouls_drawn': stat['fouls']['drawn'],
            'fouls_committed': stat['fouls']['committed'],
            'cards_yellow': stat['cards']['yellow'],
            'cards_yellowred': stat['cards']['yellowred'],
            'cards_red': stat['cards']['red'],
            'penalty_won': stat['penalty']['won'],
            'penalty_commited': stat['penalty']['commited'],
            'penalty_scored': stat['penalty']['scored'],
            'penalty_missed': stat['penalty']['missed'],
            'penalty_saved': stat['penalty']['saved']
        }
        players_info_list.append(player_info)

# Convertire la lista di dizionari in un DataFrame
players_df = pd.DataFrame(players_info_list)

# Visualizzare il DataFrame
players_df

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
0,188,S. Handanovič,Samir,Handanovič,39.0,1984-07-14,Ljubljana,Slovenia,Slovenia,193 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
1,2897,Kim Min-Jae,Min-Jae,Kim,27.0,1996-11-15,Tongyeong,Korea Republic,Korea Republic,190 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
2,30786,S. Sturaro,Stefano,Sturaro,30.0,1993-03-09,Sanremo,Italy,Italy,181 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
3,30974,A. Cordaz,Alex,Cordaz,40.0,1983-01-01,Vittorio Veneto,Italy,Italy,188 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
4,30996,M. Rohdén,Marcus Christer,Rohdén,32.0,1991-05-11,Värnamo,Sweden,Sweden,182 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,349195,E. Muhameti,Endri,Muhameti,19.0,2004-07-12,None,Italy,Albania,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
213,353610,L. Vorlický,Lukáš,Vorlický,21.0,2002-01-18,None,Czechia,Czechia,186 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
214,365734,M. Ciucci,Mattia,Ciucci,21.0,2002-11-11,Roma,Italy,Italy,188 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
215,384664,M. Fiorenza,Matteo,Fiorenza,20.0,2003-06-15,None,Italy,Italy,195 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN


In [42]:
players_df.tail()

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
212,349195,E. Muhameti,Endri,Muhameti,19.0,2004-07-12,None,Italy,Albania,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
213,353610,L. Vorlický,Lukáš,Vorlický,21.0,2002-01-18,None,Czechia,Czechia,186 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
214,365734,M. Ciucci,Mattia,Ciucci,21.0,2002-11-11,Roma,Italy,Italy,188 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
215,384664,M. Fiorenza,Matteo,Fiorenza,20.0,2003-06-15,None,Italy,Italy,195 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
216,392639,F. Raffaelli,Francesco,Raffaelli,18.0,2005-05-21,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN


In [44]:
# Filtrare i giocatori che hanno almeno una apparizione in partita
players_with_appearances = players_df.query("games_appearences > 0")
players_with_appearances

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
15,291780,D. Boloca,Daniel,Boloca,26.0,1998-12-22,Chieri,Italy,Romania,188 cm,...,7.0,25.0,6.0,0.0,1.0,None,None,0.0,0.0,NaN
17,315026,Y. Kallon,Yayah,Kallon,23.0,2001-06-30,None,Sierra Leone,Sierra Leone,175 cm,...,NaN,NaN,0.0,0.0,0.0,None,None,0.0,0.0,NaN
23,150,Arthur,Arthur Henrique,Ramos de Oliveira Melo,28.0,1996-08-12,Goiânia,Brazil,Brazil,171 cm,...,47.0,18.0,2.0,0.0,0.0,None,None,1.0,0.0,NaN
26,319,S. Luperto,Sebastiano,Luperto,28.0,1996-09-06,Lecce,Italy,Italy,191 cm,...,18.0,25.0,4.0,0.0,0.0,None,None,0.0,0.0,NaN
32,30490,N. Sansone,Nicola,Domenico Sansone,33.0,1991-09-10,München,Germany,Italy,175 cm,...,12.0,16.0,4.0,0.0,0.0,None,None,0.0,0.0,NaN
35,30981,R. Marchizza,Riccardo,Marchizza,26.0,1998-03-26,Roma,Italy,Italy,185 cm,...,3.0,6.0,1.0,0.0,0.0,None,None,0.0,0.0,NaN
39,31402,F. Ranocchia,Filippo,Ranocchia,23.0,2001-05-14,Perugia,Italy,Italy,178 cm,...,8.0,11.0,3.0,0.0,0.0,None,None,0.0,0.0,NaN
42,37437,A. Harroui,Abdoulrahmane,Harroui,26.0,1998-01-13,Leiden,Netherlands,Morocco,182 cm,...,16.0,18.0,2.0,0.0,0.0,None,None,1.0,0.0,NaN
55,22174,A. Tameze,Adrien Fidele,Tameze Aoutsa,30.0,1994-02-04,Lille,France,France,180 cm,...,15.0,19.0,6.0,0.0,1.0,None,None,0.0,0.0,NaN
57,30418,P. Gollini,Pierluigi,Gollini,29.0,1995-03-18,Bologna,Italy,Italy,194 cm,...,1.0,NaN,0.0,0.0,0.0,None,None,0.0,0.0,2.0


In [45]:
# Filtrare i giocatori il cui team_name in lowercase contiene "milan"
milan_players = players_df[players_df['team_name'].str.lower().str.contains('milan')]

# Visualizzare i giocatori filtrati
milan_players

,player_id,player_name,player_firstname,player_lastname,player_age,player_birth_date,player_birth_place,player_birth_country,player_nationality,player_height,...,fouls_drawn,fouls_committed,cards_yellow,cards_yellowred,cards_red,penalty_won,penalty_commited,penalty_scored,penalty_missed,penalty_saved
28,1831,A. Rebić,Ante,Rebić,30.0,1993-09-21,Split,Croatia,Croatia,185 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
91,277175,A. Jungdal,Andreas Kristoffer,Jungdal,21.0,2002-02-22,Singapore,Singapore,Denmark,195 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
103,81012,D. Vásquez,Devis Estiven,Vásquez Llach,25.0,1998-05-12,Barranquilla,Colombia,Colombia,195 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
107,215976,N. Michelis,Nikolaos,Michelis,22.0,2001-03-23,Nea Smyrni,Greece,Greece,192 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
111,343571,M. Malaspina,Mattia,Malaspina,18.0,2005-07-24,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
112,382947,H. Cuenca,Hugo Francisco,Cuenca Martínez,18.0,2005-01-08,None,Paraguay,Paraguay,183 cm,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
113,384419,D. Paloschi,Dorian,Paloschi,18.0,2005-11-21,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
114,384671,J. Longhi,Jordan,Longhi,18.0,2005-03-21,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
115,419617,V. Magni,Vittorio,Magni,17.0,2006-06-01,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
116,427335,D. Simic,Dario,Simic,NaN,None,None,Italy,Italy,None,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
